This is a tutorial how the features are extracted and introduces the entire data and its features:
This are for basic understanding and trial of simple models. It doesn't show the result of the best complicated models.

In [2]:
import preprocessAudioFeat as paf
#import keras_models as km
import pandas as pd

In [4]:
df=pd.DataFrame(columns=["Feature","model","Context","Accuracy","Recall","Precision","F1"])
df.to_csv('Results_4_Coria_S3.csv')
df=pd.read_csv('Results_4_Coria_S3.csv',index_col=0, header=0)


In [45]:
df.columns.tolist()

['Feature', 'model', 'Context', 'Accuracy', 'Recall', 'Precision', 'F1']

In [47]:
df = df.append(pd.Series([1,1,1,1,1,1,1], index=df.columns.tolist()), ignore_index=True)

In [38]:
df1=pd.DataFrame([[1,1,1,1,1,1,1,1]])

In [48]:
df

,Feature,model,Context,Accuracy,Recall,Precision,F1
0,1,1,1,1,1,1,1


In [60]:
ls = [1,2,3]

ls.insert(0, ls)

In [58]:
ls=["vggis"]+["td_lstm"]+ls

In [55]:
ls=[ls[0],ls[3]]+ls

In [61]:
ls

[[...], 1, 2, 3]

In [3]:
x_train1, x_test1, y_train1, y_test1=paf.loadDataFeaures('tempo')
x_train1.shape, x_test1.shape, y_train1.shape, y_test1.shape

tempo loaded
(444,) (111,)


((333, 384, 2000), (111, 384, 2000), (333,), (111,))

In [4]:
x_train, x_test, y_train, y_test=paf.loadDataFeaures('vggish')
x_train.shape, x_test.shape, y_train.shape, y_test.shape

(444,) (111,)
Vggish loaded


((333, 138, 128), (111, 138, 128), (333,), (111,))

In [8]:
x_train11, x_val,y_train11,y_val=paf.split_Data(x_train, y_train)

x_train12, x_val1,y_train12,y_val1=paf.split_Data(x_train, y_train)

(333,) (84,)


In [13]:
x_train111, x_val11,y_train111,y_val11=paf.split_Data(x_train1, y_train1)

x_train112, x_val112,y_train112,y_val112=paf.split_Data(x_train1, y_train1)

(333,) (84,)
(333,) (84,)


# Metrics used to compute recall, precision, F1 and accuracy of the methods. Thay are also provided in the libarary of this project

In [ ]:
from sklearn.metrics import recall_score,precision_score,f1_score
def rec_pre_f1_MRS(y_test,pred_test):
    rec=recall_score(y_test.tolist(),pred_test,average=None)
    pre=precision_score(y_test.tolist(),pred_test,average=None)
    f1=f1_score(y_test,pred_test,average=None)
    return rec,pre,f1

In [3]:
def rec_pre_f1(y_test,pred_test,avg='micro'):

    rec=recall_score(y_test,pred_test,average=avg,pos_label=1,labels=[1])

    prec=precision_score(y_test,pred_test,average=avg,pos_label=1,labels=[1])

    f1=f1_score(y_test,pred_test,average=avg,pos_label=1,labels=[1])
    return rec,prec,f1

def rec_pre_f1_MRS(y_test,pred_test):
    rec=recall_score(y_test.tolist(),pred_test,average=None)
    pre=precision_score(y_test.tolist(),pred_test,average=None)
    f1=f1_score(y_test,pred_test,average=None)
    return rec,pre,f1

In [8]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

dataset_Df=pd.read_csv("Scene_Dataset_Normalized.csv")
sceneLabels=dataset_Df.MRS.tolist()
labels=[i if i==0 else 1 for i in sceneLabels]
pos_mrs=[i for i in range(len(sceneLabels)) if sceneLabels[i]!=0]
mfcc,melspectogram,tempo,spec_centroids,vggFeat=paf.loadAllFeat()

In [49]:
#mfccA,melspectogramA,tempoA,spec_centroidsA,vggFeatA=loadAugmentedData()

In [10]:
indices = np.arange(len(mfcc))
x1, x2, y1, y2, idx1, idx2 = train_test_split(mfcc, labels, indices, test_size=0.25, random_state = 42)

In [55]:
#idx1

In [62]:
labels[39:41]

[0, 1]

In [56]:
posMRS_tr=[]
for i in idx1:
    if labels[i]==1:
        posMRS_tr.append(i)

In [58]:
print(posMRS_tr,len(posMRS_tr))

[382, 109, 333, 341, 103, 167, 416, 271, 209, 360, 37, 265, 146, 129, 336, 183, 233, 136, 197, 258, 432, 133, 186, 142, 240, 338, 41, 27, 347, 230, 288, 40, 375, 216, 295, 251, 309, 259, 53, 1, 366, 264, 166, 426, 54, 440, 344, 313, 413, 257, 130, 214, 443, 348] 54


Audio data simple augmentation techniques used to augment only the MRS.

In [11]:
import librosa
import soundfile

In [12]:
def manipulateNoiseInjection(data, noise_factor):
    noise = np.random.randn(len(data))
    augmented_data = data + noise_factor * noise
    # Cast back to same data type
    augmented_data = augmented_data.astype(type(data[0]))
    return augmented_data

def manipulateShiftingTime(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift    
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

def manipulateChangingPitch(data, sampling_rate, pitch_factor):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

def manipulateChangingSpeed(data, speed_factor):
    return librosa.effects.time_stretch(data, speed_factor)

In [66]:
audioDir="/vol/work3/berhe/MRS_Detection/SceneAudio/"

In [ ]:
def augmentMRS(posMRS_training)
    audioDir="/vol/work3/berhe/MRS_Detection/SceneAudio/"
    for i in posMRS_training:
        #print("Scene :{}".format(i+1))
        sys.stdout.write('Scene %s\r' % str(i+1))
        sys.stdout.flush()
        audioFile=audioDir+"Scene_"+str(i+1)+".wav"
        y, sr=librosa.load(audioFile)
        augmented_data=manipulateNoiseInjection(y,noise_factor=0.005)
        soundfile.write(audioDir+"AugmentedMRS/Noise_Scene_"+str(i+1)+".wav", augmented_data, sr)
        augmented_data=manipulateChangingSpeed(y,speed_factor=2)
        soundfile.write(audioDir+"AugmentedMRS/Speed_Scene_"+str(i+1)+".wav", augmented_data, sr)
        augmented_data=manipulateShiftingTime(y,sampling_rate=sr, shift_max=30, shift_direction="left")
        soundfile.write(audioDir+"AugmentedMRS/ShiftLeft_Scene_"+str(i+1)+".wav", augmented_data, sr)
        augmented_data=manipulateChangingPitch(y,sampling_rate=sr,pitch_factor=5)
        soundfile.write(audioDir+"AugmentedMRS/Pitch_Scene_"+str(i+1)+".wav", augmented_data, sr)
    print("finished")

In [100]:
testLabels=[]
for i in range(idx2.shape[0]):
    testLabels.append(labels[idx2[i]])
testLabels=np.array(testLabels)

In [99]:
idx2[1]

137

Saving the split

In [16]:
x_train, x_test, y_train, y_test=km.processDataB(vggFeat,labels,testSize=0.25)
pickle.dump(x_train,open("Data/TrainingData/vggish_q_x_training","wb"))
pickle.dump(y_train,open("Data/TrainingData/vggish_q_y_train","wb"))
pickle.dump(x_test,open("Data/TestData/vggish_q_x_test","wb"))
pickle.dump(y_test,open("Data/TestData/vggish_q_y_test","wb"))

(444,) (111,)


In [14]:
len(x_train), len(x_test)

(333, 111)

In [23]:
paf.generateSameLength?

In [13]:
mfcc_sl=paf.generateSameLength(mfcc,)
mel_sl=paf.generateSameLength(melspectogram)
mfcc_aug,binaryLabels=paf.augmentMRS_data(pos_mrs,mfcc,labels)
mfcc_r=paf.augment_reshape(mfcc_aug)
mel_aug,binaryLabels=paf.augmentMRS_data(pos_mrs,melspectogram,labels)
mel_r=paf.augment_reshape(mel_aug)

In [88]:
y_test

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
def sameLengthVgg(vggfeat):
    sceneLength=[i.shape[0] for i in vggfeat]
    minLength=min(sceneLength)
    print(minLength)
    reshapedData=[]
    for i in vggfeat:
        reshapedData.append(i[i.shape[0]-minLength:i.shape[0],:])
    reshapedData=np.array(reshapedData)
    return reshapedData

In [17]:
def augment_reshapeVgg(audioData,avgLength=138):
    augmentZero=np.zeros(audioData[0].shape[1])
    sceneLength=[i.shape[0] for i in audioData]
    minLength=min(sceneLength)
    reshapedData=[]
    for i in audioData:
        dataArray=i
        if dataArray.shape[0] < avgLength:
            for j in range(dataArray.shape[0]+1,avgLength+1):
                dataArray=np.concatenate((dataArray,augmentZero[None,:]),axis=0)
            print(dataArray.shape,dataArray[::-1].shape)
            reshapedData.append(dataArray)
        else:
            print("here",i[i.shape[0]-avgLength:,:].shape)
            reshapedData.append(i[i.shape[0]-avgLength:,:])
    reshapedData=np.array(reshapedData)
    return reshapedData

In [59]:
vggFeat_sl_a=sameLengthVgg(vggFeatA)

13


In [60]:
vggFeat_sl_a.shape

(216, 13, 128)

In [18]:
mfcc_r.shape

(660, 20, 2000)

In [37]:
vggFeat[0][vggFeat[0].shape[0]-20:vggFeat[0].shape[0],:].shape

(20, 128)

In [33]:
vggFeat_sl.shape,vggFeat[0].shape,mfcc[0].shape

((444, 1, 128), (92, 128), (20, 3112))

# MFCC features results

In [19]:
x_train, x_test, y_train, y_test=km.processDataB(mfcc_r,binaryLabels,testSize=0.33)
print(x_train.shape,x_test.shape)

(660,) (218,)
(442, 20, 2000) (218, 20, 2000)


In [20]:
pred_test_r, y_test=km.trainModel(mfcc_r,binaryLabels,epochs=5,batch_size=64,model_type='cnn')
rec,prec,f1=rec_pre_f1_MRS(y_test,pred_test_r)
print(rec[1],prec[1],f1[1])

(660,) (165,)
(165, 20, 2000) (495, 20, 2000)

(165, 20, 2000, 1) (165,)

Train on 495 samples, validate on 165 samples
Epoch 1/5
 - 8s - loss: 3.3711 - accuracy: 0.4949 - val_loss: 0.7743 - val_accuracy: 0.4727

Epoch 00001: val_loss improved from inf to 0.77431, saving model to weights.best.cnn.hdf5
Epoch 2/5
 - 7s - loss: 1.1380 - accuracy: 0.5131 - val_loss: 1.0528 - val_accuracy: 0.4364

Epoch 00002: val_loss did not improve from 0.77431
Epoch 3/5
 - 7s - loss: 0.7981 - accuracy: 0.4970 - val_loss: 0.7600 - val_accuracy: 0.4545

Epoch 00003: val_loss improved from 0.77431 to 0.75996, saving model to weights.best.cnn.hdf5
Epoch 4/5
 - 7s - loss: 0.7699 - accuracy: 0.5293 - val_loss: 0.9521 - val_accuracy: 0.4303

Epoch 00004: val_loss did not improve from 0.75996
Epoch 5/5
 - 7s - loss: 0.7100 - accuracy: 0.5717 - val_loss: 0.7926 - val_accuracy: 0.4303

Epoch 00005: val_loss did not improve from 0.75996
Training completed in time:  0:00:38.008737
Training Accuracy 0.45858585834503

In [21]:
pred_test_r, y_test=km.trainModel(mfcc_r,binaryLabels,epochs=5,batch_size=64,model_type='lstm')
rec,prec,f1=rec_pre_f1_MRS(y_test,pred_test_r)
print(rec[1],prec[1],f1[1])

(660,) (165,)
Build LSTM RNN model ...
(165, 20, 2000) (165,)
Train on 495 samples, validate on 165 samples
Epoch 1/5
 - 6s - loss: 0.7040 - accuracy: 0.5535 - val_loss: 0.6713 - val_accuracy: 0.5879

Epoch 00001: val_loss improved from inf to 0.67134, saving model to weights.best.lstm.hdf5
Epoch 2/5
 - 4s - loss: 0.4642 - accuracy: 0.8141 - val_loss: 0.6665 - val_accuracy: 0.6364

Epoch 00002: val_loss improved from 0.67134 to 0.66654, saving model to weights.best.lstm.hdf5
Epoch 3/5
 - 4s - loss: 0.3359 - accuracy: 0.9172 - val_loss: 0.7295 - val_accuracy: 0.6000

Epoch 00003: val_loss did not improve from 0.66654
Epoch 4/5
 - 4s - loss: 0.2373 - accuracy: 0.9212 - val_loss: 0.7885 - val_accuracy: 0.6242

Epoch 00004: val_loss did not improve from 0.66654
Epoch 5/5
 - 4s - loss: 0.2102 - accuracy: 0.9293 - val_loss: 0.8555 - val_accuracy: 0.6364

Epoch 00005: val_loss did not improve from 0.66654
Training completed in time:  0:00:26.349687
Training Accuracy 0.9353535175323486  
Testi

In [22]:
pred_test_s, y_test_s=km.trainModel(mfcc_sl,labels,epochs=5,batch_size=64,model_type='cnn')
rec,prec,f1=rec_pre_f1_MRS(y_test_s,pred_test_s)
print(rec[1],prec[1],f1[1])

(444,) (111,)
(111, 20, 61) (333, 20, 61)
(111, 20, 61, 1) (111,)
Train on 333 samples, validate on 111 samples
Epoch 1/5
 - 1s - loss: 5.4456 - accuracy: 0.5736 - val_loss: 2.5085 - val_accuracy: 0.8378

Epoch 00001: val_loss improved from inf to 2.50853, saving model to weights.best.cnn.hdf5
Epoch 2/5
 - 0s - loss: 3.7218 - accuracy: 0.8378 - val_loss: 1.7968 - val_accuracy: 0.8378

Epoch 00002: val_loss improved from 2.50853 to 1.79685, saving model to weights.best.cnn.hdf5
Epoch 3/5
 - 0s - loss: 2.0393 - accuracy: 0.8288 - val_loss: 0.6102 - val_accuracy: 0.8378

Epoch 00003: val_loss improved from 1.79685 to 0.61018, saving model to weights.best.cnn.hdf5
Epoch 4/5
 - 0s - loss: 1.1147 - accuracy: 0.6757 - val_loss: 0.5360 - val_accuracy: 0.7568

Epoch 00004: val_loss improved from 0.61018 to 0.53602, saving model to weights.best.cnn.hdf5
Epoch 5/5
 - 0s - loss: 0.7368 - accuracy: 0.7177 - val_loss: 0.4381 - val_accuracy: 0.8378

Epoch 00005: val_loss improved from 0.53602 to 0.43

In [48]:
print(rec,prec,f1)

[1. 0.] [0.8707483 0.       ] [0.93090909 0.        ]


In [10]:
pred_test_s, y_test_s=km.trainModel(mfcc_sl,labels,epochs=5,batch_size=64,model_type='lstm')
rec,prec,f1=rec_pre_f1_MRS(y_test_s,pred_test_s)
print(rec[1],prec[1],f1[1])
#print(rec,prec,f1)

(444,) (147,)
Build LSTM RNN model ...
(147, 20, 61) (147,)
Train on 297 samples, validate on 147 samples
Epoch 1/5
 - 2s - loss: 0.5456 - accuracy: 0.7811 - val_loss: 0.4000 - val_accuracy: 0.8707

Epoch 00001: val_loss improved from inf to 0.40004, saving model to weights.best.lstm.hdf5
Epoch 2/5
 - 1s - loss: 0.4558 - accuracy: 0.8215 - val_loss: 0.3945 - val_accuracy: 0.8707

Epoch 00002: val_loss improved from 0.40004 to 0.39445, saving model to weights.best.lstm.hdf5
Epoch 3/5
 - 1s - loss: 0.4251 - accuracy: 0.8249 - val_loss: 0.4055 - val_accuracy: 0.8707

Epoch 00003: val_loss did not improve from 0.39445
Epoch 4/5
 - 1s - loss: 0.3950 - accuracy: 0.8283 - val_loss: 0.3998 - val_accuracy: 0.8707

Epoch 00004: val_loss did not improve from 0.39445
Epoch 5/5
 - 1s - loss: 0.3606 - accuracy: 0.8316 - val_loss: 0.3935 - val_accuracy: 0.8707

Epoch 00005: val_loss improved from 0.39445 to 0.39354, saving model to weights.best.lstm.hdf5
Training completed in time:  0:00:09.060175
Tr

In [13]:
import numpy as np
nsamples, nx, ny = mfcc_sl.shape
d2_train_dataset = mfcc_sl.reshape((nsamples,nx*ny))

pred_test_svm_sl, y_test_svm=km.predictSVM(d2_train_dataset,labels)
y_test_svm=np.array(y_test_svm)
rec,prec,f1=rec_pre_f1_MRS(y_test_svm,pred_test_svm_sl)
print(rec[1],prec[1],f1[1])

Accuracy Score : 0.81:
[1. 0.] [0.80952381 0.        ] [0.89473684 0.        ]


In [15]:
nsamples, nx, ny = mfcc_r.shape
d2_train_dataset = mfcc_r.reshape((nsamples,nx*ny))

pred_test_svm_r, y_test_svm=km.predictSVM(d2_train_dataset,binaryLabels)
y_test_svm=np.array(y_test_svm)
rec,prec,f1=rec_pre_f1_MRS(y_test_svm,pred_test_r)
print(rec[1],prec[1],f1[1])

Accuracy Score : 0.638:
0.3978494623655914 0.4157303370786517 0.4065934065934066


# MelSpectogram Features

In [28]:
pred_test_s_mel, y_test_s_mel=km.trainModel(mel_sl,labels,epochs=5,batch_size=64,model_type='cnn')
y_test_s_mel=np.array(y_test_s_mel)
rec,prec,f1=rec_pre_f1_MRS(y_test_s_mel,pred_test_s_mel)
print(rec[1],prec[1],f1[1])

(444,) (147,)
(147, 128, 61, 1) (147,)
Train on 297 samples, validate on 147 samples
Epoch 1/5
 - 2s - loss: 0.6362 - accuracy: 0.6835 - val_loss: 0.6558 - val_accuracy: 0.8707

Epoch 00001: val_loss improved from inf to 0.65577, saving model to weights.best.cnn.hdf5
Epoch 2/5
 - 1s - loss: 0.6114 - accuracy: 0.8215 - val_loss: 0.6699 - val_accuracy: 0.8707

Epoch 00002: val_loss did not improve from 0.65577
Epoch 3/5
 - 1s - loss: 0.5909 - accuracy: 0.8215 - val_loss: 0.6244 - val_accuracy: 0.8707

Epoch 00003: val_loss improved from 0.65577 to 0.62439, saving model to weights.best.cnn.hdf5
Epoch 4/5
 - 1s - loss: 0.5564 - accuracy: 0.8215 - val_loss: 0.5809 - val_accuracy: 0.8707

Epoch 00004: val_loss improved from 0.62439 to 0.58092, saving model to weights.best.cnn.hdf5
Epoch 5/5
 - 1s - loss: 0.5442 - accuracy: 0.8215 - val_loss: 0.5545 - val_accuracy: 0.8707

Epoch 00005: val_loss improved from 0.58092 to 0.55452, saving model to weights.best.cnn.hdf5
Training completed in time:

In [30]:
pred_test_s_mel, y_test_s_mel=km.trainModel(mel_sl,labels,epochs=5,batch_size=64,model_type='lstm')
y_test_s_mel=np.array(y_test_s_mel)
rec,prec,f1=rec_pre_f1_MRS(y_test_s_mel,pred_test_s_mel)
print(rec[1],prec[1],f1[1])

(444,) (147,)
Build LSTM RNN model ...
(147, 128, 61) (147,)
Train on 297 samples, validate on 147 samples
Epoch 1/5
 - 7s - loss: 0.6739 - accuracy: 0.7609 - val_loss: 0.5906 - val_accuracy: 0.8707

Epoch 00001: val_loss improved from inf to 0.59063, saving model to weights.best.lstm.hdf5
Epoch 2/5
 - 6s - loss: 0.5417 - accuracy: 0.8215 - val_loss: 0.3848 - val_accuracy: 0.8707

Epoch 00002: val_loss improved from 0.59063 to 0.38484, saving model to weights.best.lstm.hdf5
Epoch 3/5
 - 6s - loss: 0.4759 - accuracy: 0.8249 - val_loss: 0.4089 - val_accuracy: 0.8707

Epoch 00003: val_loss did not improve from 0.38484
Epoch 4/5
 - 6s - loss: 0.4795 - accuracy: 0.8182 - val_loss: 0.3890 - val_accuracy: 0.8707

Epoch 00004: val_loss did not improve from 0.38484
Epoch 5/5
 - 6s - loss: 0.4662 - accuracy: 0.8249 - val_loss: 0.3921 - val_accuracy: 0.8707

Epoch 00005: val_loss did not improve from 0.38484
Training completed in time:  0:00:35.020170
Training Accuracy 0.8249158263206482  
Testin

In [31]:
pred_test_r_mel, y_test_r_mel=km.trainModel(mel_r,binaryLabels,epochs=5,batch_size=64,model_type='cnn')
#y_test_s_mel=np.array(y_test_s_mel)
rec,prec,f1=rec_pre_f1_MRS(y_test_r_mel,pred_test_r_mel)
print(rec[1],prec[1],f1[1])

(660,) (218,)
(218, 128, 2000, 1) (218,)
Train on 442 samples, validate on 218 samples
Epoch 1/5
 - 63s - loss: 0.6979 - accuracy: 0.5204 - val_loss: 0.6920 - val_accuracy: 0.5229

Epoch 00001: val_loss improved from inf to 0.69202, saving model to weights.best.cnn.hdf5
Epoch 2/5
 - 61s - loss: 0.6901 - accuracy: 0.5543 - val_loss: 0.6981 - val_accuracy: 0.5229

Epoch 00002: val_loss did not improve from 0.69202
Epoch 3/5
 - 63s - loss: 0.6892 - accuracy: 0.5566 - val_loss: 0.6983 - val_accuracy: 0.5321

Epoch 00003: val_loss did not improve from 0.69202
Epoch 4/5
 - 65s - loss: 0.6914 - accuracy: 0.5566 - val_loss: 0.6932 - val_accuracy: 0.5275

Epoch 00004: val_loss did not improve from 0.69202
Epoch 5/5
 - 71s - loss: 0.6928 - accuracy: 0.5204 - val_loss: 0.7107 - val_accuracy: 0.5183

Epoch 00005: val_loss did not improve from 0.69202
Training completed in time:  0:05:29.384793
Training Accuracy 0.5633484125137329  
Testing Accuracy 0.5183486342430115 
0.0 0.0 0.0


In [33]:
pred_test_r_mel, y_test_r_mel=km.trainModel(mel_r,binaryLabels,epochs=5,batch_size=64,model_type='lstm')
#y_test_s_mel=np.array(y_test_s_mel)
rec,prec,f1=rec_pre_f1_MRS(y_test_r_mel,pred_test_r_mel)
print(rec[1],prec[1],f1[1])

(660,) (218,)
Build LSTM RNN model ...
(218, 128, 2000) (218,)
Train on 442 samples, validate on 218 samples
Epoch 1/5
 - 27s - loss: 0.6912 - accuracy: 0.5656 - val_loss: 0.6736 - val_accuracy: 0.5780

Epoch 00001: val_loss improved from inf to 0.67360, saving model to weights.best.lstm.hdf5
Epoch 2/5
 - 26s - loss: 0.6182 - accuracy: 0.6584 - val_loss: 0.7442 - val_accuracy: 0.5459

Epoch 00002: val_loss did not improve from 0.67360
Epoch 3/5
 - 25s - loss: 0.5364 - accuracy: 0.7330 - val_loss: 0.7547 - val_accuracy: 0.5275

Epoch 00003: val_loss did not improve from 0.67360
Epoch 4/5
 - 25s - loss: 0.4642 - accuracy: 0.7851 - val_loss: 0.8123 - val_accuracy: 0.5826

Epoch 00004: val_loss did not improve from 0.67360
Epoch 5/5
 - 24s - loss: 0.3923 - accuracy: 0.8281 - val_loss: 0.9413 - val_accuracy: 0.5826

Epoch 00005: val_loss did not improve from 0.67360
Training completed in time:  0:02:14.719580
Training Accuracy 0.8823529481887817  
Testing Accuracy 0.5825688242912292 
0.4838

In [35]:
nsamples, nx, ny = mel_sl.shape
d2_train_dataset = mel_sl.reshape((nsamples,nx*ny))

pred_test_svm_sl_mel, y_test_svm=km.predictSVM(d2_train_dataset,labels)
y_test_svm=np.array(y_test_svm)
rec,prec,f1=rec_pre_f1_MRS(y_test_svm,pred_test_svm_sl_mel)
print(rec[1],prec[1],f1[1])

Accuracy Score : 0.81:
0.0 0.0 0.0


In [38]:
nsamples, nx, ny = mel_r.shape
d2_train_dataset = mel_r.reshape((nsamples,nx*ny))
pred_test_svm_r_mel, y_test_svm_aug=km.predictSVM(d2_train_dataset,binaryLabels)
y_test_svm=np.array(y_test_svm_aug)
rec,prec,f1=rec_pre_f1_MRS(y_test_svm,pred_test_svm_r_mel)
print(rec[1],prec[1],f1[1])

Accuracy Score : 0.624:
0.3010752688172043 0.6222222222222222 0.4057971014492754


In [37]:
y_test_svm=np.array(y_test_svm_aug)
rec,prec,f1=rec_pre_f1_MRS(y_test_svm,pred_test_svm_r_mel)
print(rec[1],prec[1],f1[1])

0.3010752688172043 0.6222222222222222 0.4057971014492754


# VGGish Features

In [56]:
pred_test_r, y_test=km.trainModel(vggFeat_aug,labels,epochs=5,batch_size=64,model_type='cnn')
rec,prec,f1=rec_pre_f1_MRS(y_test,pred_test_r)
print(rec[1],prec[1],f1[1])

(444,) (147,)

(147, 138, 128, 1) (147,)

Train on 297 samples, validate on 147 samples
Epoch 1/5
 - 3s - loss: 0.5856 - accuracy: 0.8215 - val_loss: 0.5016 - val_accuracy: 0.8707

Epoch 00001: val_loss improved from inf to 0.50159, saving model to weights.best.cnn.hdf5
Epoch 2/5
 - 2s - loss: 0.4756 - accuracy: 0.8215 - val_loss: 0.4312 - val_accuracy: 0.8707

Epoch 00002: val_loss improved from 0.50159 to 0.43124, saving model to weights.best.cnn.hdf5
Epoch 3/5
 - 2s - loss: 0.5031 - accuracy: 0.8215 - val_loss: 0.4291 - val_accuracy: 0.8707

Epoch 00003: val_loss improved from 0.43124 to 0.42915, saving model to weights.best.cnn.hdf5
Epoch 4/5
 - 2s - loss: 0.4817 - accuracy: 0.8215 - val_loss: 0.4615 - val_accuracy: 0.8707

Epoch 00004: val_loss did not improve from 0.42915
Epoch 5/5
 - 2s - loss: 0.4784 - accuracy: 0.8215 - val_loss: 0.4815 - val_accuracy: 0.8707

Epoch 00005: val_loss did not improve from 0.42915
Training completed in time:  0:00:14.355785
Training Accuracy 0.821

In [64]:
pred_test_r, y_test=km.trainModel(vggFeat_augT,labels,epochs=5,batch_size=64,model_type='lstm')
rec,prec,f1=rec_pre_f1_MRS(y_test,pred_test_r)
print(rec[1],prec[1],f1[1])

(444,) (147,)
Build LSTM RNN model ...
(147, 128, 138) (147,)
Train on 297 samples, validate on 147 samples
Epoch 1/5
 - 7s - loss: 0.5283 - accuracy: 0.7744 - val_loss: 0.4130 - val_accuracy: 0.8707

Epoch 00001: val_loss improved from inf to 0.41303, saving model to weights.best.lstm.hdf5
Epoch 2/5
 - 6s - loss: 0.4751 - accuracy: 0.8215 - val_loss: 0.4175 - val_accuracy: 0.8707

Epoch 00002: val_loss did not improve from 0.41303
Epoch 3/5
 - 6s - loss: 0.4547 - accuracy: 0.8215 - val_loss: 0.3941 - val_accuracy: 0.8707

Epoch 00003: val_loss improved from 0.41303 to 0.39412, saving model to weights.best.lstm.hdf5
Epoch 4/5
 - 6s - loss: 0.4457 - accuracy: 0.8215 - val_loss: 0.4004 - val_accuracy: 0.8707

Epoch 00004: val_loss did not improve from 0.39412
Epoch 5/5
 - 6s - loss: 0.4450 - accuracy: 0.8215 - val_loss: 0.4177 - val_accuracy: 0.8707

Epoch 00005: val_loss did not improve from 0.39412
Training completed in time:  0:00:35.010143
Training Accuracy 0.8215488195419312  
Testi

In [65]:
pred_test_r, y_test=km.trainModel(vggFeat_augT,labels,epochs=5,batch_size=64,model_type='cnn')
rec,prec,f1=rec_pre_f1_MRS(y_test,pred_test_r)
print(rec[1],prec[1],f1[1])

(444,) (147,)
(147, 128, 138, 1) (147,)
Train on 297 samples, validate on 147 samples
Epoch 1/5
 - 3s - loss: 0.5859 - accuracy: 0.8215 - val_loss: 0.4995 - val_accuracy: 0.8707

Epoch 00001: val_loss improved from inf to 0.49955, saving model to weights.best.cnn.hdf5
Epoch 2/5
 - 2s - loss: 0.4798 - accuracy: 0.8215 - val_loss: 0.4306 - val_accuracy: 0.8707

Epoch 00002: val_loss improved from 0.49955 to 0.43063, saving model to weights.best.cnn.hdf5
Epoch 3/5
 - 2s - loss: 0.4954 - accuracy: 0.8215 - val_loss: 0.4387 - val_accuracy: 0.8707

Epoch 00003: val_loss did not improve from 0.43063
Epoch 4/5
 - 2s - loss: 0.4783 - accuracy: 0.8215 - val_loss: 0.4720 - val_accuracy: 0.8707

Epoch 00004: val_loss did not improve from 0.43063
Epoch 5/5
 - 2s - loss: 0.4780 - accuracy: 0.8215 - val_loss: 0.4814 - val_accuracy: 0.8707

Epoch 00005: val_loss did not improve from 0.43063
Training completed in time:  0:00:14.630325
Training Accuracy 0.8215488195419312  
Testing Accuracy 0.8707482814

In [25]:
#5 Frames VGGish
feat=np.load("5_frames.npy")

In [31]:
x_train, x_test, y_train, y_test=km.processDataB(feat,labels,testSize=0.25)

(444,) (111,)


In [32]:
x_train.shape

(333, 62, 128)

In [33]:
pred_test_r, y_test=km.trainModel_splitedData(x_train, x_test, y_train, y_test,epochs=5,batch_size=64,model_type='lstm')
rec,prec,f1=rec_pre_f1_MRS(y_test,pred_test_r)
print(rec[1],prec[1],f1[1])

Build LSTM RNN model ...
(333, 62, 128) (111, 62, 128)
Train on 333 samples, validate on 111 samples
Epoch 1/5
 - 9s - loss: 0.5634 - accuracy: 0.7057 - val_loss: 0.4760 - val_accuracy: 0.8378
Epoch 2/5
 - 7s - loss: 0.4565 - accuracy: 0.8378 - val_loss: 0.4435 - val_accuracy: 0.8378
Epoch 3/5
 - 3s - loss: 0.4481 - accuracy: 0.8378 - val_loss: 0.4445 - val_accuracy: 0.8378
Epoch 4/5
 - 6s - loss: 0.4525 - accuracy: 0.8378 - val_loss: 0.4441 - val_accuracy: 0.8378
Epoch 5/5
 - 7s - loss: 0.4444 - accuracy: 0.8378 - val_loss: 0.4433 - val_accuracy: 0.8378
Training completed in time:  0:00:37.681847
Training Accuracy 0.837837815284729  
Testing Accuracy 0.837837815284729 
0.0 0.0 0.0


In [34]:
pred_test_r, y_test=km.trainModel_splitedData(x_train, x_test, y_train, y_test,epochs=5,batch_size=64,model_type='cnn')
rec,prec,f1=rec_pre_f1_MRS(y_test,pred_test_r)
print(rec[1],prec[1],f1[1])

(111, 62, 128) (333, 62, 128)
(333, 62, 128, 1) (111, 62, 128, 1)
Train on 333 samples, validate on 111 samples
Epoch 1/5
 - 4s - loss: 25.3319 - accuracy: 0.5676 - val_loss: 10.7397 - val_accuracy: 0.8378
Epoch 2/5
 - 1s - loss: 17.4319 - accuracy: 0.8378 - val_loss: 6.3996 - val_accuracy: 0.8378
Epoch 3/5
 - 1s - loss: 7.9188 - accuracy: 0.8378 - val_loss: 6.5741 - val_accuracy: 0.1622
Epoch 4/5
 - 1s - loss: 3.2765 - accuracy: 0.4474 - val_loss: 0.9946 - val_accuracy: 0.8378
Epoch 5/5
 - 2s - loss: 2.7601 - accuracy: 0.8378 - val_loss: 0.4925 - val_accuracy: 0.8378
Training completed in time:  0:00:14.955924
Training Accuracy 0.837837815284729  
Testing Accuracy 0.837837815284729 
0.0 0.0 0.0


# Augmented Audio

In [37]:
x_train1=np.average(x_train,axis=1)
x_test1=np.average(x_test,axis=1)
pred_test,y_test=km.predictSVMSplit(x_train1, x_test1, y_train, y_test)

Accuracy Score : 0.838:


In [38]:
rec,prec,f1=rec_pre_f1_MRS(y_test,pred_test)
print(rec[1],prec[1],f1[1])

0.0 0.0 0.0


In [39]:
x_train1.shape

(333, 128)

In [41]:
x_test1.shape

(111, 128)